In this notebook I've tried a new strategy, I specified for each row the time, starting from the initial time_to_eruption time associated with each dataset in order to perform a regression model on each row. I'm open to suggetion about this new strategy :)

I used just 500 out of the 4000 available dataset cause otherwise it crashs. 

In [ ]:
!pip install ProgressBar

In [ ]:
import numpy as np 
import pandas as pd 
from progressbar import ProgressBar
import glob
import os

In [ ]:
frag = glob.glob("../input/predict-volcanic-eruptions-ingv-oe/train/*")

In [ ]:
def preprocessing_train(name):
    train_df = pd.read_csv('../input/predict-volcanic-eruptions-ingv-oe/train.csv')
    dataframe = pd.DataFrame()
    pbar = ProgressBar()
    index=[]
    for i in range(0,len(frag[0:5])):
        index = np.append(index,os.path.splitext(frag[i].split('{}/'.format(name))[1])[0])
    index = index.astype('int')

    for i in pbar(index):
        df = pd.read_csv('../input/predict-volcanic-eruptions-ingv-oe/{}/{}.csv'.format(name,i))
        value = (train_df['time_to_eruption'][train_df['segment_id']==i])
        value = np.array(value)
        valuelist = np.arange(value,value+60001,1)
        df['time_to_eruption']=valuelist
        dataframe = pd.concat([dataframe,df],axis=0)
        del df
    return(dataframe)

def preprocessing_test(name):
    index = []
    frag = glob.glob("../input/predict-volcanic-eruptions-ingv-oe/{}/*".format(name))
    df=pd.DataFrame()

    pbar = ProgressBar()
    for i in pbar(frag):
        df = np.append(df,pd.read_csv(i).mean())
    
    df = pd.DataFrame(df.reshape(len(frag),10))  

    for i in range(0,len(frag)):
        index = np.append(index,os.path.splitext(frag[i].split('{}/'.format(name))[1])[0])
        
    df['segment_id']=index
    df['segment_id']=df['segment_id'].astype(int)
    df.columns= ['sensor_1', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5', 'sensor_6', 'sensor_7', 'sensor_8', 'sensor_9', 'sensor_10','segment_id']
    return(df)

def fill_na(data):
    for i in data.columns:
        data[i] = data[i].fillna(np.mean(data[i]))

In [ ]:
%%time
data_x = preprocessing_train('train')
data_y= preprocessing_test('test')

In [ ]:
for i in range(0,60001):
    print(i,data_x.iloc[i,-1])

In [ ]:
fill_na(data_x)
fill_na(data_y)

In [ ]:
y = data_x['time_to_eruption']
x= data_x.iloc[:,0:-1]

del data_x

In [ ]:
%%time 

from  sklearn.model_selection import train_test_split
import lightgbm as lgb

Xt, Xv, Yt, Yv = train_test_split(x, y, test_size =0.2, shuffle=False)

params = {
        'objective': 'regression', #specify how is the dependet variable, binary can be used for logistic regression or log loss classification
        'max_bin': 600, #max number of bins that features values will be bucketed in. Small number may reduce training accuracy but may increase general power
        'learning_rate': 0.02, #learning_rate refers to the step size at each interation while moving toward an optimal point
        'num_leaves': 80, # maximum number of leaves in a tree, where a leave is a final termination of a tree
        'metric' : 'mae'
}


lgb_train = lgb.Dataset(Xt, Yt)
lgb_eval = lgb.Dataset(Xv, Yv, reference=lgb_train)
        #lightgbm need to take as argument lightgbm dataset, it is required to make this trasformation

model = lgb.train(
            params, lgb_train, #it is required to insert the parameters, then the train set
            valid_sets=[lgb_train, lgb_eval],
            verbose_eval=100,
            num_boost_round=1500, # number of boosting iterations 
            early_stopping_rounds=15 # will stop training if one metric of one validation data doesn’t improve in last early_stopping_round rounds, so if 
            #  for ten 'epochs' the model will stop, in this way the num_boost_round is a maximum value.  
)  

y_pred = model.predict(Xv)
y_true = np.array(Yv)
print('mean absolute error:',mae(y_true, y_pred))

prediction = model.predict(data_y.iloc[:,0:-1])

In [ ]:
sub_df = pd.DataFrame(data_y['segment_id'])
sub_df = pd.concat([sub_df,pd.Series(prediction)],axis=1)
sample_submission=pd.read_csv('../input/predict-volcanic-eruptions-ingv-oe/sample_submission.csv')
        
sample_submission = pd.merge(sample_submission,sub_df, on =['segment_id'])
sample_submission = sample_submission.drop(columns=['time_to_eruption'])
sample_submission.columns = ['segment_id', 'time_to_eruption']
sample_submission.to_csv('sample_submission.csv', header=True, index=False)
print('saved :)')